In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell

In [2]:
embedding=np.random.rand(5,5)   #random embedding matrix

In [3]:
#sentences=np.random.randint(5, size=(10, 2))  #Each sentence is of lenght 2
sentences=np.array([[1,2,3],[3,1],[1,2,3,4],[2,3],[0,2]])
labels=np.random.randint(2,size=(10,))   #Labels 0,1 for each sentence
mapping={0:(1,0),1:(0,1)}
labels=[mapping[label] for label in labels]   #Creating the labels
#sentences,labels
print np.shape(embedding[sentences[1]])   #shape is n_steps,n_input..What is expected?

(2, 5)


In [4]:
# Parameters
learning_rate = 0.001
training_iters = 10000
display_step = 100
batch_size = 1
# Network Parameters
n_input = 5 #size of each word vector
#n_steps = 2 # timesteps in each sample
n_hidden = 10 # hidden layer num of features
n_classes = 2 # MNIST total classes (0-9 digits)

In [8]:
y = tf.placeholder("float", [None, n_classes])
istate = tf.placeholder("float", [None, 2*n_hidden]) #state & cell => 2x n_hidden

cost = 0
count  = 0
for sent in sentences:
    count += 1
    print count
    num_steps=len(sent)
    x = tf.placeholder("float", [None, num_steps, n_input])
    pred = RNN(x, istate)

# Define loss and optimizer
    cost += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
    
    
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    
    

1


ValueError: Variable RNN/BasicLSTMCell/Linear/Matrix already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-6-69a450e33275>", line 26, in RNN
    outputs, states = rnn.rnn(lstm_cell, X, initial_state=_istate)
  File "<ipython-input-7-430682e51fec>", line 7, in <module>
    pred = RNN(x, istate)
  File "/Users/abhimanyu/tensorflow/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# tf Graph input
#x = tf.placeholder("float", [None, n_steps, n_input])
# x = tf.placeholder("float", [None, n_steps, n_input])   #Number of steps can be variable
# istate = tf.placeholder("float", [None, 2*n_hidden]) #state & cell => 2x n_hidden
# y = tf.placeholder("float", [None, n_classes])


# # Define weights
# weights = {
#     'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
#     'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
# }
# biases = {
#     'hidden': tf.Variable(tf.random_normal([n_hidden])),
#     'out': tf.Variable(tf.random_normal([n_classes]))
# }
with tf.variable_scope('weights'):
    hidden_weights=tf.get_variable('hidden',shape=(n_input, n_hidden))
    out_weights=tf.get_variable('out',shape=(n_hidden, n_classes))
with tf.variable_scope('biases'):
    hidden_biases=tf.get_variable('hidden',shape=(n_hidden))
    out_biases=tf.get_variable('out',shape=(n_classes))
        
    

In [6]:
def RNN(X, _istate):

    with tf.variable_scope('weights',reuse=True):
        hidden_weights=tf.get_variable('hidden',shape=(n_input, n_hidden))
        out_weights=tf.get_variable('out',shape=(n_hidden, n_classes))
    with tf.variable_scope('biases',reuse=True):
        hidden_biases=tf.get_variable('hidden',shape=(n_hidden))
        out_biases=tf.get_variable('out',shape=(n_classes))
        
    
    # input shape: (batch_size, n_steps, n_input)
    shape=X.get_shape().as_list()  #Gets the shape of the tensor
    X = tf.transpose(X, [1, 0, 2])  # permute n_steps and batch_size
    
    # Reshape to prepare input to hidden activation
    X = tf.reshape(X, [-1, n_input]) # (n_steps*batch_size, n_input)
    # Linear activation
    X = tf.matmul(X, hidden_weights) + hidden_biases

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=0.9)
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    X = tf.split(0, shape[1], X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, X, initial_state=_istate)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], out_weights) + out_biases

In [9]:
pred = RNN(x, istate, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

NameError: name 'x' is not defined

In [7]:

def get_next_batch(step):
    total_sent=len(sentences)
    sent=sentences[step%total_sent]   #returns this sentence
    x_data=embedding[sent]
    y_data=labels[step%total_sent]
    return x_data,np.array(y_data)
    
    
        
        
        
        
    

In [8]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 0
    # Keep training until reach max iterations
    y = tf.placeholder("float", [None, n_classes])
    istate = tf.placeholder("float", [None, 2*n_hidden]) #state & cell => 2x n_hidden
    while step * batch_size < training_iters:
        batch_xs, batch_ys = get_next_batch(step)
        
        num_steps=np.shape(batch_xs)[0]
        
#         print np.shape(batch_xs),np.shape(batch_ys)
#         break
#         batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))
#         print np.shape(batch_xs)
#         print np.shape(batch_ys)
        
        # Reshape data to get 28 seq of 28 elements
        batch_xs = batch_xs.reshape((batch_size, num_steps, n_input))
        batch_ys=batch_ys.reshape((batch_size,n_classes))
#         print np.shape(batch_xs)
#         print np.shape(batch_ys)
#         break
        # Fit training using batch data
        x = tf.placeholder("float", [None, num_steps, n_input])
        pred = RNN(x, istate)

# Define loss and optimizer
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
    
    
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Evaluate model
        correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys,
                                       istate: np.zeros((batch_size, 2*n_hidden))})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys,
                                                istate: np.zeros((batch_size, 2*n_hidden))})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,
                                             istate: np.zeros((batch_size, 2*n_hidden))})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
#     test_len = 256
#     test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
#     test_label = mnist.test.labels[:test_len]
#     print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label,
#                                                              istate: np.zeros((test_len, 2*n_hidden))})

FailedPreconditionError: Attempting to use uninitialized value RNN/BasicLSTMCell/Linear/Matrix
	 [[Node: RNN/BasicLSTMCell/Linear/Matrix/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](RNN/BasicLSTMCell/Linear/Matrix)]]
Caused by op u'RNN/BasicLSTMCell/Linear/Matrix/read', defined at:
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Library/Python/2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/abhimanyu/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Library/Python/2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start(

In [ ]:
a=tf.constant([[1,2,3],[3,4,5]])
a.get_shape().as_list()[1].dtype

In [ ]:
sentences
